# Find the synthetic data following the electrochemical theory

In [98]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
import os, sys 
import pandas as pd
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns
from PEMFC_project import _samplings , _equivalent_circuits, _reconstruct_Nyquist_plot,_data_summery 

import pickle

# Load Gaussian process classifier which has been trained from original dataset (EIS_O2_STOI.csv) 
with open('Models/gpc_smote_3.pkl','rb') as f:
    gpc = pickle.load(f)
    
def GPC(x) :
    o2_stoi = [1.4, 1.5, 1.8, 2.0, 2.2, 2.5, 3.0]
    y_c = gpc.predict(x)
    return np.array([o2_stoi[i] for i in y_c.astype(int)])

C:\Users\mero_\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator _BinaryGaussianProcessClassifierLaplace from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\mero_\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\mero_\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 0.23.2 when using version 1.2.2. This might lead to breaking code or inv

In [99]:
def box_plot_stats(
    
    ## arguments / inputs
    x,          ## input array of values 
    coef = 1.5  ## positive real number
                ## (determines how far the whiskers extend from the iqr)
    ):          
    
    ## quality check for coef
    if coef <= 0:
        raise ValueError("cannot proceed: coef must be greater than zero")
    
    ## convert input to numpy array
    x = np.array(x)
    
    ## determine median, lower ‘hinge’, upper ‘hinge’
    median = np.quantile(a = x, q = 0.50, method = "midpoint")
    first_quart = np.quantile(a = x, q = 0.25, method = "midpoint")
    third_quart = np.quantile(a = x, q = 0.75, method = "midpoint")
    
    ## calculate inter quartile range
    intr_quart_rng = third_quart - first_quart
    
    ## calculate extreme of the lower whisker (observed, not interpolated)
    lower = first_quart - (coef * intr_quart_rng)
    lower_whisk = np.compress(x >= lower, x)
    lower_whisk_obs = np.min(lower_whisk)
    
    ## calculate extreme of the upper whisker (observed, not interpolated)
    upper = third_quart + (coef * intr_quart_rng)
    upper_whisk = np.compress(x <= upper, x)
    upper_whisk_obs = np.max(upper_whisk)
    
    ## store box plot results dictionary
    boxplot_stats = {}
    boxplot_stats["stats"] = np.array([lower_whisk_obs, 
                                       first_quart, 
                                       median, 
                                       third_quart, 
                                       upper_whisk_obs])
   
    ## store observations beyond the box plot extremes
    boxplot_stats["xtrms"] = np.array(x[(x < lower_whisk_obs) | 
                                        (x > upper_whisk_obs)])
    
    ## return dictionary        
    return boxplot_stats

In [100]:
def drop_ex(data1, data2, ind):
    ind_d1 = np.where(box_plot_stats(data1)['xtrms'])[0]
    ind_d2 = np.where(box_plot_stats(data2)['xtrms'])[0]
    ind_d3 = np.where(data1<1e-15)[0]
    ind_d4 = np.where(data2<1e-4)[0]
    ind_d_ = np.unique(np.r_[ind_d1, ind_d2,ind_d3,ind_d4]) 
    return np.delete(ind, ind_d_)

def drop_ex_2(data1, data2, ind):
    ind_d1 = np.where(data1 < box_plot_stats(data1)['stats'][0])[0]
    ind_d2 = np.where(data1 > box_plot_stats(data1)['stats'][4])[0]
    ind_d3 = np.where(data2 < box_plot_stats(data2)['stats'][0])[0]
    ind_d4 = np.where(data2 > box_plot_stats(data2)['stats'][4])[0]
    ind_d_ = np.unique(np.r_[ind_d1, ind_d2, ind_d3, ind_d4]) 
    return np.delete(ind, ind_d_)



In [101]:
df_param1 = pd.read_csv('Data/kde_param_7.csv',header=0)
df_zf1 = pd.read_csv('Data/kde_zf_7.csv',header=0)
df_param2 = pd.read_csv('Data/kde_param_6.csv',header=0)
df_zf2 = pd.read_csv('Data/kde_zf_6.csv',header=0)

In [102]:
df_param = pd.concat([df_param1, df_param2],ignore_index=False)

df_zf= pd.concat([df_zf1, df_zf2],ignore_index=False)


df_zf.reset_index(drop=True, inplace=True)

df_param.reset_index(drop=True, inplace=True)

df_zf.drop(index=np.where((df_param.iloc[:,0])<1e-11)[0],inplace=True)
df_param.drop(index=np.where((df_param.iloc[:,0])<1e-11)[0],inplace=True)
df_zf.reset_index(drop=True, inplace=True)
df_param.reset_index(drop=True, inplace=True)

In [103]:
eis_train = pd.read_csv('Data/x_smote_enn_train_1.csv',header=0,index_col=None)

scaler =MinMaxScaler()
scaler.fit(eis_train.values)
zf1 = scaler.transform(df_zf.iloc[:,:-1].values)
y_zf1 = gpc_se3.predict(zf1)
target = pd.Series(y_zf1).replace([0, 1, 2, 3, 4, 5, 6],[1.4, 1.5,  1.8, 2.0, 2.2, 2.5, 3.0])
df_param['O2 stoi'] = target.values
j_lim = {1.4:10800, 1.5:11600,1.6:12200, 1.8:13700, 2.0:15100, 2.2:16500, 2.5:18700, 3.0:22200}
j_lim_p = [j_lim[x] for x in df_param['O2 stoi']]
df_param['J lim'] = j_lim_p

df_param['(Rct/Jlim)^2'] = (df_param['Rct']/df_param['J lim'])**2

In [104]:
ind_d1 = df_param.loc[df_param['(Rct/Jlim)^2']<1e-14,'(Rct/Jlim)^2'].index
ind_d2 = df_param.loc[df_param['Rt']<1e-4,'Rt'].index
ind_d = np.unique(np.r_[ind_d1, ind_d2])

df_param.drop(index=ind_d,inplace=True)
df_zf.drop(index=ind_d,inplace=True)

df_zf.reset_index(drop=True, inplace=True)
df_param.reset_index(drop=True, inplace=True)
_data_summery.target_summery(df_param['O2 stoi'])

,counts,proportion
O2 stoi,,
1.8,921,0.333092
1.5,470,0.169982
1.4,393,0.142134
2.0,388,0.140325
2.2,318,0.115009
2.5,142,0.051356
3.0,133,0.048101


In [68]:
ind_sel = np.array([])
stois = [1.5,1.8,2.0,2.2,2.5,3.0]
np.random.default_rng()

for stoi in stois:
    ind = np.where(df_param.loc[:,'O2 stoi'] == stoi)[0]
    ind_sel = np.r_[ind_sel, np.random.choice(ind,size=120, replace=False)].astype('int')

df_p_sel = df_param.iloc[ind_sel,:]
df_z_sel = df_zf.iloc[ind_sel,:]


df_p_sel.reset_index(drop=True, inplace=True)
df_z_sel.reset_index(drop=True, inplace=True)

target_count = _data_summery.target_summery(df_p_sel['O2 stoi'])
target_count.sort_index()

,counts,proportion
O2 stoi,,
1.5,120,0.166667
1.8,120,0.166667
2.0,120,0.166667
2.2,120,0.166667
2.5,120,0.166667
3.0,120,0.166667


In [115]:


stois = [1.5,1.8,2.0,2.2,2.5,3.0]
np.random.default_rng()

R2_score = 0

i = 0

while R2_score<0.99 and i < 200:
#if i == 0:
    
    i += 1
    ind_sel = np.array([])
    for stoi in stois:
        ind = np.where(df_param.loc[:,'O2 stoi'] == stoi)[0]
        ind_sel = np.r_[ind_sel, np.random.choice(ind,size=120, replace=False)].astype('int')

    df_p_sel = df_param.iloc[ind_sel,:]
    df_z_sel = df_zf.iloc[ind_sel,:]


    df_p_sel.reset_index(drop=True, inplace=True)
    df_z_sel.reset_index(drop=True, inplace=True)

    ind_sel2 = np.array([])

    for stoi in O2stoi:
        ind = np.where(df_p_sel['O2 stoi']==stoi)[0]
        data1 = df_p_sel.loc[df_p_sel['O2 stoi']==stoi,'(Rct/Jlim)^2']
        data2 = df_p_sel.loc[df_p_sel['O2 stoi']==stoi,'Rt']
        ex1 = box_plot_stats(data1)['xtrms']
        ex2 = box_plot_stats(data2)['xtrms']
        ind_sel2 = np.unique(np.r_[ind_sel2, drop_ex_2(data1, data2, ind)])
    
    df_p_sel2 = df_p_sel.iloc[ind_sel2.astype(int),:]
    df_z_sel2=df_z_sel.iloc[ind_sel2.astype(int),:]
    df_p_sel2.reset_index(drop=True, inplace=True)
    df_z_sel2.reset_index(drop=True, inplace=True)
    
    df_m = df_p_sel2.groupby('O2 stoi').mean()
    df_std = df_p_sel2.groupby('O2 stoi').std()

    X_m = ((df_m['Rct']*28.3/(df_m['J lim']*1e-4))**2).values.reshape(-1,1)
    y_m = df_m['Rt']*28.3

    reg = LinearRegression().fit(X_m , y_m)
    R2_score = reg.score(X_m,y_m)
    
    

In [116]:
df_p_sel2 = df_p_sel.iloc[ind_sel2.astype(int),:]
df_z_sel2=df_z_sel.iloc[ind_sel2.astype(int),:]
df_p_sel2.reset_index(drop=True, inplace=True)
df_z_sel2.reset_index(drop=True, inplace=True)
_data_summery.target_summery(df_p_sel2['O2 stoi'])

,counts,proportion
O2 stoi,,
1.8,118,0.173785
1.5,115,0.169367
2.0,113,0.166421
2.2,113,0.166421
2.5,113,0.166421
3.0,107,0.157585


In [117]:
df_m = df_p_sel2.groupby('O2 stoi').mean()
df_std = df_p_sel2.groupby('O2 stoi').std()

X_m = ((df_m['Rct']*28.3/(df_m['J lim']*1e-4))**2).values.reshape(-1,1)
y_m = df_m['Rt']*28.3

reg = LinearRegression().fit(X_m , y_m)
R2_score = reg.score(X_m,y_m)
print('R2=%0.4f'%R2_score)
print('y=%0.4f*(R_ct/J_lim)**2+%0.4f'%(reg.coef_[0],reg.intercept_))

R2=0.9947
y=6.2703*(R_ct/J_lim)**2+0.0467
